![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [292]:
import pandas as pd
import numpy as np

# Start coding here...

In [293]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


# Processing for _client.csv_

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

Quickly view the info about the given `df` then get the required columns from `df` and store into **`clients_df`**

In [294]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [295]:
clients_df = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]
clients_df.head()


,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


## client.csv: Check if there are missing values

In [296]:
print(clients_df.info(),'\n')
clients_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   mortgage        41188 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB
None 



client_id         0
age               0
job               0
marital           0
education         0
credit_default    0
mortgage          0
dtype: int64

There are a total of 41,188 rows, and there appears to be no missing values for the client-related columns, so we can skip the handling of missing data.

## client.csv: Sanity checking for columns w/o cleaning requirements
Of the seven (7) required columns for `client.csv`, there are three (3) without any cleaning requiements:
- `client_id`
- `age`
- `marital`

### client.csv: client_id
As a sanity check, the `client_id` column should:
- contain 41,888 unique values
- if incrementing, minimum value should be 0 while maximum should be 41187

Checking below, no further processing is required for `client_id` .


In [297]:
print(f"number of unique client_id: {len(clients_df['client_id'].unique())}")
print(f"min client_id: {clients_df['client_id'].min()}, \
      max client_id: {clients_df['client_id'].max()}")

number of unique client_id: 41188
min client_id: 0,       max client_id: 41187


### client.csv: age
As a sanity check, the `age` column should:
- be positive integers greater than 0

Checking below, the youngest client is 17 years old while the oldest is 98. No further processing is required for `age` .

In [298]:
print(clients_df['age'].min(), clients_df['age'].max())

17 98


### client.csv: marital
As a sanity check, the `marital` column should:
- be uniform in terms of case
- not have trailing or leading whitespaces

Checking below, there are four (4) unique values in the column.
Although the 'unknown' value is somewhat alarming, we will just leave it as-is.

No further processing is required for `marital` .

In [299]:
print(f"unique values for marital: {clients_df['marital'].unique()}.")

unique values for marital: ['married' 'single' 'divorced' 'unknown'].


## client.csv: Clean the columns with specified requirements
The following columns have specific cleaning requirements
- `job`
- `education`
- `credit_default`
- `mortgage`

Each column is tackled in each sub-section.


### client.csv: `job`
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |

Let's take a peek at what kind of values have an offending `.` character:

In [300]:
clients_job_withdot = clients_df['job'].str.contains('\.')
print(f"number of job values with '.': {clients_job_withdot.sum()}")
clients_df[ clients_job_withdot ]['job'].unique()

number of job values with '.': 10422


array(['admin.'], dtype=object)

We found 10,422 entries with a `job` value of "admin.". Let's do a simple replace for the offending rows.

Afterwards, we can do a quick check afterwards that there are no more offending rows.

In [301]:
clients_df.loc[ clients_job_withdot, 'job'] = clients_df[ clients_job_withdot ]['job'].str.replace("\.", "_")

# check that there are no more "education" values with "." character
clients_job_withdot = clients_df['job'].str.contains('\.')
assert(clients_job_withdot.sum() == 0)

### client.csv: education
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |

Taking a peek at the offending rows with "." characters:

In [302]:
clients_educ_withdot = clients_df['education'].str.contains('\.')
print(f"number of education values with '.': {clients_educ_withdot.sum()}")
clients_df[ clients_educ_withdot ]['education'].unique()

number of education values with '.': 39439


array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', 'university.degree'], dtype=object)

We found 39,439 entries with a `education` value with the "." character. Let's do a simple replace for the offending rows.

Afterwards, we can do a quick check afterwards that there are no more offending rows.

In [303]:
clients_df.loc[ clients_educ_withdot, 'education'] = clients_df[ clients_educ_withdot ]['education'].str.replace("\.", "_")

# check that there are no more "education" values with "." character
clients_educ_withdot = clients_df['education'].str.contains('\.')
assert(clients_educ_withdot.sum() == 0)

Next, let's change the "unknown" values to `np.NaN`.
Let's quickly view first the unique values in the column and check the count of offending rows.

In [304]:
print(f"unique education values: {clients_df['education'].unique()}\n")

client_educ_unknown = clients_df['education'] == "unknown"
num_client_educ_unknown = client_educ_unknown.sum()
print(f"number of \"unknown\" education values: {num_client_educ_unknown}")


unique education values: ['basic_4y' 'high_school' 'basic_6y' 'basic_9y' 'professional_course'
 'unknown' 'university_degree' 'illiterate']

number of "unknown" education values: 1731


There are 1,731 rows to change. Let's proceed with the replacement and assert that there are no more offending rows.

Then we're all done with the `education` column.

In [305]:
clients_df.loc[ client_educ_unknown, 'education'] = clients_df[ client_educ_unknown ]['education'].replace("unknown", np.nan)

# check that there are no more "unknown" "education" values,
# and that there is the same number of null values as there were previously unknown values
client_educ_unknown = clients_df['education'] == "unknown"
assert(client_educ_unknown.sum() == 0)
assert( clients_df['education'].isna().sum() == num_client_educ_unknown )

### client.csv: credit_default
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
 
Let's take a peek at the unique 'credit_default' values and the number of entries with each value:

In [306]:
client_creditdefault_counts_df = clients_df['credit_default'].value_counts()
client_creditdefault_counts_df

no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64

There are three (3) unique values ("yes", "no", "unknown") with their counts shown and stored above.

Let's do the replacement as follows:
* "yes" -> 1
* "no", "unknown" -> 0

Afterwards do some checks on the counts. Then we're done with the `credit_default` column

In [307]:
client_creditdefault_is_yes = clients_df['credit_default'] == "yes"
clients_df.loc[client_creditdefault_is_yes, 'credit_default'] = 1
clients_df.loc[~client_creditdefault_is_yes, 'credit_default'] = 0
clients_df['credit_default'] = clients_df['credit_default'].astype(bool)

# Check that number of 'True' 'credit_default' rows is equal to the number of rows previously "yes",
# and that the number of 'False' rows is equal to the sum of "no"s and "unknowns" previously
assert((clients_df['credit_default'] == True).sum() == client_creditdefault_counts_df["yes"])
assert((clients_df['credit_default'] == False).sum() ==
       (client_creditdefault_counts_df["no"] + client_creditdefault_counts_df["unknown"]))

### client.csv: mortgage
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

In [308]:
client_mortgage_counts_df = clients_df['mortgage'].value_counts()
client_mortgage_counts_df

yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64

The `mortgage` column is in the same situation with the `credit_default` in the previous section, so let's apply the same process.

Then we're done with the 'mortgage' column.

In [309]:
clients_df['mortgage'] = clients_df['mortgage'].replace(to_replace = {"yes":1, "no":0, "unknown":0}).astype(bool)

# Check that number of 'True' 'mortgage' rows is equal to the number of rows previously "yes",
# and that the number of 'False' rows is equal to the sum of "no"s and "unknowns" previously
assert((clients_df['mortgage'] == True).sum() == client_mortgage_counts_df["yes"])
assert((clients_df['mortgage'] == False).sum() ==
       (client_mortgage_counts_df["no"] + client_mortgage_counts_df["unknown"]))

## client.csv: Export to csv
At this point, all our clean client data is stored in `clients_df`, and all that's left to do is to export it to a CSV file.

In [310]:
clients_df.to_csv('client.csv')

In [311]:
# Check first few rows of exported CSV file
with open('client.csv') as csvfile:
    for i in range(0,12):
        print(csvfile.readline())

,client_id,age,job,marital,education,credit_default,mortgage

0,0,56,housemaid,married,basic_4y,False,False

1,1,57,services,married,high_school,False,False

2,2,37,services,married,high_school,False,True

3,3,40,admin_,married,basic_6y,False,False

4,4,56,services,married,high_school,False,False

5,5,45,services,married,basic_9y,False,False

6,6,59,admin_,married,professional_course,False,False

7,7,41,blue-collar,married,,False,False

8,8,24,technician,single,professional_course,False,True

9,9,25,services,single,high_school,False,True

10,10,41,blue-collar,married,,False,False



# Processing for campaign.csv

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

Quickly view the info about the given `df` then get the required columns from `df` and store into **`campaign_df`**

In [312]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [313]:
campaign_df = df[['client_id', 'number_contacts', 'contact_duration',
                'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'month', 'day']]
campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
0,0,1,261,0,nonexistent,no,may,13
1,1,1,149,0,nonexistent,no,may,19
2,2,1,226,0,nonexistent,no,may,23
3,3,1,151,0,nonexistent,no,may,27
4,4,1,307,0,nonexistent,no,may,3


## campaign.csv: Check if there are missing values

In [314]:
print(campaign_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  object
 5   campaign_outcome            41188 non-null  object
 6   month                       41188 non-null  object
 7   day                         41188 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.5+ MB
None


In [315]:
campaign_df.isna().sum()

client_id                     0
number_contacts               0
contact_duration              0
previous_campaign_contacts    0
previous_outcome              0
campaign_outcome              0
month                         0
day                           0
dtype: int64

There are a total of 41,188 rows, and there appears to be no missing values for the campaign-related columns, so we can skip the handling of missing data.

## campaign.csv: Sanity checking for columns w/o cleaning requirements
Of the seven (7) required columns for `campaign.csv`, there are four (4) without any cleaning requiements:
- `client_id`
- `number_contacts`
- `contact_duration`
- `previous_campaign_contacts`

### campaign.csv: client_id
As a sanity check, the `client_id` column should:
- contain 41,888 unique values
- if incrementing, minimum value should be 0 while maximum should be 41187

Checking below, no further processing is required for `client_id` .

In [316]:
print(f"number of unique client_id: {len(clients_df['client_id'].unique())}")
print(f"min client_id: {clients_df['client_id'].min()}, \
      max client_id: {clients_df['client_id'].max()}")

number of unique client_id: 41188
min client_id: 0,       max client_id: 41187


### campaign.csv: number_contacts
As a sanity check, the `number_contacts` column should:
- be positive integers greater than 0

No further processing is required for `number_contacts` .

In [317]:
print(campaign_df['number_contacts'].min(), campaign_df['number_contacts'].max())

1 56


### campaign.csv: contact_duration
As a sanity check, the `contact_duration` column should:
- be positive integers greater than 0

No further processing is required for `contact_duration` .

In [318]:
print(campaign_df['contact_duration'].min(), campaign_df['contact_duration'].max())

0 4918


### campaign.csv: previous_campaign_contacts
As a sanity check, the `previous_campaign_contacts` column should:
- be positive integers greater than 0

No further processing is required for `previous_campaign_contacts` .

In [319]:
print(campaign_df['previous_campaign_contacts'].min(), campaign_df['previous_campaign_contacts'].max())

0 7


## campaign.csv: Clean the columns with specified requirements
The following columns have specific cleaning and generating requirements
- `previous_outcome`
- `campaign_outcome`
- `last_contact_date`

Each column is tackled in each sub-section.


### campaign.csv: previous_outcome
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |

In [320]:
campaign_prevoutcome_counts_df = campaign_df['previous_outcome'].value_counts()
campaign_prevoutcome_counts_df

nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64

There are three (3) unique values ("success", "failure", "nonexistent") with their counts shown and stored above.

Let's do the replacement as follows:
* "success" -> 1
* "failure", "nonexistent" -> 0

Afterwards do some checks on the counts. Then we're done with the 'previous_outcome' column

In [321]:
campaign_prevoutcome_is_success = campaign_df['previous_outcome'] == "success"
campaign_df.loc[campaign_prevoutcome_is_success, 'previous_outcome'] = 1
campaign_df.loc[~campaign_prevoutcome_is_success, 'previous_outcome'] = 0
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].astype(bool)

# Check that number of 'True' 'previous_outcome' rows is equal to the number of rows previously "yes",
# and that the number of 'False' rows is equal to the sum of "no"s and "unknowns" previously
assert((campaign_df['previous_outcome'] == True).sum() == campaign_prevoutcome_counts_df["success"])
assert((campaign_df['previous_outcome'] == False).sum() ==
       (campaign_prevoutcome_counts_df["nonexistent"] + campaign_prevoutcome_counts_df["failure"]))

### campaign.csv: campaign_outcome
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |

In [322]:
campaign_campoutcome_counts_df = campaign_df['campaign_outcome'].value_counts()
campaign_campoutcome_counts_df

no     36548
yes     4640
Name: campaign_outcome, dtype: int64

In [323]:
campaign_campoutcome_is_yes = campaign_df['campaign_outcome'] == "yes"
campaign_df.loc[campaign_campoutcome_is_yes, 'campaign_outcome'] = 1
campaign_df.loc[~campaign_campoutcome_is_yes, 'campaign_outcome'] = 0
campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].astype(bool)

# Check that number of 'True' 'previous_outcome' rows is equal to the number of rows previously "yes",
# and that the number of 'False' rows is equal to the sum of "no"s and "unknowns" previously
assert((campaign_df['campaign_outcome'] == True).sum() == campaign_campoutcome_counts_df["yes"])
assert((campaign_df['campaign_outcome'] == False).sum() == (campaign_campoutcome_counts_df["no"]))

### campaign.csv: last_contact_date
| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

In [324]:
print(f"Unique months in campaign_df: {campaign_df['month'].unique()}")
print(f"Min and Max days in campaign_df: {campaign_df['day'].min()}, {campaign_df['day'].max()}")

Unique months in campaign_df: ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
Min and Max days in campaign_df: 1, 31


In [325]:
# Get the month and day as-is from campaign_df
last_contact_date_df = campaign_df[['month', 'day']]

# Set the year to 2022 for all entries
last_contact_date_df['year'] = 2022

# Convert month string to number
month_dict = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6,
              'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12,}
last_contact_date_df['month'] = last_contact_date_df['month'].replace(to_replace=month_dict).astype(int)

In [326]:
last_contact_date_df.head()

,month,day,year
0,5,13,2022
1,5,19,2022
2,5,23,2022
3,5,27,2022
4,5,3,2022


In [327]:
campaign_df['last_contact_date'] = pd.to_datetime(arg=last_contact_date_df, format="%Y-%m-%d")
campaign_df = campaign_df.drop(columns = ['month', 'day'])

In [328]:
# Check that all values in 'last_contact_date' are dates
# in between 2022-01-01(bounded) and 2023-01-01(unbounded)
assert( ((campaign_df['last_contact_date'] >= "2022-01-01") & 
         (campaign_df['last_contact_date'] < "2023-01-01")).sum() == len(campaign_df) )

# Check if month and day columns are not present anymore
assert( ("month" not in campaign_df.columns) and ("day" not in campaign_df.columns) )

campaign_df['last_contact_date'].head(), campaign_df['last_contact_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 41188 entries, 0 to 41187
Series name: last_contact_date
Non-Null Count  Dtype         
--------------  -----         
41188 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 321.9 KB


(0   2022-05-13
 1   2022-05-19
 2   2022-05-23
 3   2022-05-27
 4   2022-05-03
 Name: last_contact_date, dtype: datetime64[ns],
 None)

## campaign.csv Export to csv

In [329]:
campaign_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB


In [330]:
campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03


In [331]:
campaign_df.to_csv('campaign.csv')

In [332]:
# Check first few rows of exported CSV file
with open('campaign.csv') as csvfile:
    for i in range(0,12):
        print(csvfile.readline())

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date

0,0,1,261,0,False,False,2022-05-13

1,1,1,149,0,False,False,2022-05-19

2,2,1,226,0,False,False,2022-05-23

3,3,1,151,0,False,False,2022-05-27

4,4,1,307,0,False,False,2022-05-03

5,5,1,198,0,False,False,2022-05-05

6,6,1,139,0,False,False,2022-05-03

7,7,1,217,0,False,False,2022-05-12

8,8,1,380,0,False,False,2022-05-21

9,9,1,50,0,False,False,2022-05-05

10,10,1,55,0,False,False,2022-05-08



# Processing for economics.csv

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

Quickly view the info about the given `df` then get the required columns from `df` and store into **`economics_df`**

In [333]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [334]:
economics_df = df[['client_id', 'cons_price_idx', 'euribor_three_months']]
economics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB


In [335]:
economics_df.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


In [336]:
economics_df.to_csv('economics.csv')

In [337]:
# Check first few rows of exported CSV file
with open('economics.csv') as csvfile:
    for i in range(0,12):
        print(csvfile.readline())

,client_id,cons_price_idx,euribor_three_months

0,0,93.994,4.857

1,1,93.994,4.857

2,2,93.994,4.857

3,3,93.994,4.857

4,4,93.994,4.857

5,5,93.994,4.857

6,6,93.994,4.857

7,7,93.994,4.857

8,8,93.994,4.857

9,9,93.994,4.857

10,10,93.994,4.857

